In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install u8darts
!pip install matplotlib==3.1.3

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import torch
from darts.dataprocessing.transformers import Scaler
from darts.metrics import r2_score
from darts.models.filtering.moving_average import MovingAverage
from darts.utils.timeseries_generation import datetime_attribute_timeseries
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('../input/gabung-covid-mobility/gabungSemua.csv')

In [ ]:
df = df[df['Location']=='Indonesia'].reset_index(drop=True)

In [ ]:
df.corr()

# Preprocessing

In [ ]:
pilih_kolom = ['New Cases',	'New Deaths']
df_indo = df[['Date']+pilih_kolom]

In [ ]:
df_indo['Date'] = pd.to_datetime(df_indo['Date'])

In [ ]:
df_indo['Date'][0]

In [ ]:
df.isnull().sum()

In [ ]:
line_stat = []
line_stat.append(go.Scatter(x=df['Date'],
                            y=df['New Cases'], name="New Cases Covid Indonesia", mode='lines'))

fig = go.Figure(data=line_stat)
fig.update_layout(title_text="Total Cases Covid Indonesia")
fig.show()


In [ ]:
from darts import TimeSeries
scaler = Scaler()
series = scaler.fit_transform(TimeSeries.from_dataframe(df, 'Date', pilih_kolom))

# apply moving avarage
moving_avarage = MovingAverage(10)
series = moving_avarage.filter(series)
series.plot()

# split


In [ ]:
potong_tanggal = pd.Timestamp('20210808')
pd.Timestamp('20211105') - potong_tanggal

In [ ]:
hari_test = 89

In [ ]:
# Create training and validation sets:
train, val = series.split_after(potong_tanggal)

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

# create month and year covariate series
year_series = datetime_attribute_timeseries(pd.date_range(start=series.start_time(), freq=series.freq_str, periods=1000),
                                             attribute='year', one_hot=False)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(year_series, attribute='month', one_hot=True)
covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(potong_tanggal)

# VAR Model

In [ ]:
# r2=0.5
# while r2<0.8:
#     from darts.models.forecasting.varima import VARIMA
#     model_var = VARIMA()
#     model_var.fit(train_transformed)

#     # VAR TEST
#     def eval_model(model):
#         pred_series = model.predict(n=89, future_covariates=covariates)
#         plt.figure(figsize=(8,5))
#         series_transformed.univariate_component(0).plot(label='actual')
#         pred_series.univariate_component(0).plot(label='forecast')
#         plt.title('R2: {}'.format(r2_score(pred_series.univariate_component(0), val_transformed.univariate_component(0))))
#         r2 = r2_score(pred_series.univariate_component(0), val_transformed.univariate_component(0))
#         if r2>0.8:
#             plt.legend();
#         print(r2)
#     eval_model(model_var)
    

In [ ]:
from darts.models.forecasting.varima import VARIMA
model_var = VARIMA()
model_var.fit(train_transformed)

# VAR TEST
def eval_model(model):
    pred_series = model.predict(n=89, future_covariates=covariates)
    plt.figure(figsize=(8,5))
    series_transformed.univariate_component(0).plot(label='actual')
    pred_series.univariate_component(0).plot(label='forecast')
    plt.title('R2: {}'.format(r2_score(pred_series.univariate_component(0), val_transformed.univariate_component(0))))
    r2 = r2_score(pred_series.univariate_component(0), val_transformed.univariate_component(0))
    plt.legend();
eval_model(model_var)

# RNN LSTM

In [ ]:
from darts.models.forecasting.rnn_model import RNNModel
model_lstm = RNNModel(
    model='LSTM',
    hidden_dim=64,
    dropout=0.2,
    batch_size=32,
    n_epochs=300,
    optimizer_kwargs={'lr': 1e-3},
    model_name='Air_RNN',
    log_tensorboard=True,
    random_state=1,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True)

In [ ]:
model_lstm.fit(train_transformed,
             future_covariates=covariates,
             val_series=val_transformed,
             val_future_covariates=covariates,
             verbose=True)

# Test LSTM

In [ ]:
eval_model(model_lstm)

# Forcasting

In [ ]:
pd.Timestamp('2022-01-05') - pd.Timestamp('2021-11-05')


In [ ]:
hari_prediksi = 61

In [ ]:
model_var.predict(n=hari_test+hari_prediksi).univariate_component(0).plot()
series_transformed.univariate_component(0).plot()